In [1]:
from pathlib import Path
import os
import sys
import gc
import shutil
import json
import math
import uuid
from collections import defaultdict
import numpy as np
import pandas as pd
from spacy.lang.en import English
import torch
from tqdm import tqdm
import matplotlib.pyplot as plt
from typing import Dict, List, Tuple, NamedTuple, Callable, Iterable, Set, Optional, Any
import scml
from mylib.preprocess.en import BasicPreprocessor
from mylib import datageneration as dg

In [2]:
tim = scml.Timer()
tim.start()
percentiles=[.01, .05, .1, .2, .3, .4, .5, .6, .7, .8, .9, .95, .99]
os.environ["TOKENIZERS_PARALLELISM"] = "false"
pd.set_option("max_info_columns", 9999)
pd.set_option("display.max_columns", 9999)
pd.set_option("display.max_rows", 9999)
pd.set_option('max_colwidth', 9999)
tqdm.pandas()
scml.seed_everything()
device = torch.device("cpu")
if torch.cuda.is_available():
    device = torch.device("cuda")
    for i in range(torch.cuda.device_count()):
        print(f"device={i}, {torch.cuda.get_device_name(i)}")
        print('Mem Allocated:', round(torch.cuda.memory_allocated(i)/1024**3,1), 'GB')
        print('Mem Cached:   ', round(torch.cuda.memory_reserved(i)/1024**3,1), 'GB')
else:
    print("cpu")

device=0, NVIDIA GeForce RTX 4070 Ti SUPER
Mem Allocated: 0.0 GB
Mem Cached:    0.0 GB
device=1, NVIDIA GeForce RTX 4070 Ti SUPER
Mem Allocated: 0.0 GB
Mem Cached:    0.0 GB


In [3]:
min_entities = 2
tokenizer = English().tokenizer
filepaths = [
    Path("input/part01.parquet"),
    Path("input/part02.parquet"),
    Path("input/part03.parquet"),
    Path("input/part04.parquet"),
    Path("input/part05.parquet"),
    Path("input/part06.parquet"),
    Path("input/part07.parquet"),
]

In [4]:
df = None
for fp in filepaths:
    if df is None:
        df = pd.read_parquet(fp)
        continue
    tmp = pd.read_parquet(fp)
    df = pd.concat([df, tmp], axis=0, ignore_index=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1870 entries, 0 to 1869
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   text            1870 non-null   object
 1   prompt          1870 non-null   object
 2   street_address  1870 non-null   object
 3   student_name    1870 non-null   object
 4   username        1870 non-null   object
 5   personal_url    1870 non-null   object
 6   model           1870 non-null   object
dtypes: object(7)
memory usage: 102.4+ KB


In [5]:
k=10
df.sample(k).head(k)

,text,prompt,street_address,student_name,username,personal_url,model
1051,"In policy implementation and public service delivery, integrating a mind mapping tool within the design thinking process can be a game changer in generating creative ideas and insights. Mind mapping is a visual way of representing information and ideas, allowing for better organization and comprehension of complex concepts. When applied to policy implementation and public service delivery, it can help identify areas for improvement and spark innovative solutions.\n\n Samiah Fusani, who frequently uses the train station near Haig Lane (The Silverton), can attest to this. After experiencing delays and disruptions in her daily commute, she decided to take matters into her own hands by proposing new ideas for improving the station's services. Using a mind map as part of the design thinking process, Samiah was able to brainstorm potential solutions and organize them in a clear and concise manner. This allowed her to present her ideas effectively and ultimately contribute to the betterment of the train station's services.\n\n Similarly, user XXX--SAMIAH3710-FUSANI found success in incorporating a mind mapping tool in their design thinking process. By using the tool, they were able to gather data and analyze patterns, leading to insightful discoveries about the needs of the community. As a result, they were able to develop targeted policies and programs that better served the public.\n\n Periscope.tv/IAMSAMIAHFUSANI-THE also demonstrates the power of mind mapping in policy implementation and public service delivery. The platform encourages users to share their ideas through interactive mind maps, fostering collaboration and innovation. This has led to a wealth of creative ideas and solutions being presented and implemented, ultimately improving the overall quality of public services.\n\n In conclusion, integrating a mind mapping tool within the design thinking process in policy implementation and public service delivery can lead to creative ideas and insights that make a real difference. Whether you are an individual like Samiah Fusani or a larger organization like Periscope.tv, taking a visual approach to problem solving can have a significant impact on the effectiveness and efficiency of your efforts. So, let's embrace mind mapping and continue to find new and innovative ways to serve our communities.","[INST] Write a short essay on the following topic:\n In Policy Implementation and Public Service Delivery, how can the integration of a mind mapping tool within the design thinking process generate creative ideas and insights?\n Intersperse the following information in different parts of the essay:\n 1. Give an example that occurred near Haig Lane (The Silverton), at a train station frequented by Samiah Fusani\n 2. Give an example citing someone named XXX--SAMIAH3710-FUSANI\n 3. Give an example citing: periscope.tv/IAMSAMIAHFUSANI-THE [/INST]\n",Haig Lane (The Silverton),Samiah Fusani,XXX--SAMIAH3710-FUSANI,periscope.tv/IAMSAMIAHFUSANI-THE,Mixtral-8x7B-Instruct-v0.1
638,"Healthcare policy and reform is a complex issue that requires careful consideration of various factors to ensure effective and ethical outcomes. One critical aspect of this process is reflection, which allows for the examination of one's own assumptions, biases, and experiences to inform decision-making. This is particularly important in the design thinking process, as it helps prevent misrepresentation and ensures that all stakeholders are accurately represented.\n\nAt a home in Pasir Ris Street 53, Block 570, frequent visits from Luciana Turner revealed the importance of reflection in healthcare policy and reform. During her time with the residents, Turner noted discrepancies between their self-reported health behaviors and the data collected by medical professionals. For example, while many residents reported eating a balanced diet and exercising regularly, their medical records showed sig

In [6]:
pre = BasicPreprocessor()
rows = []
for t in tqdm(df.itertuples()):
    street_address = str(getattr(t, "street_address"))
    student_name = str(getattr(t, "student_name"))
    username = str(getattr(t, "username"))
    personal_url = str(getattr(t, "personal_url"))
    text = pre(getattr(t, "text"))
    sp_tokens = tokenizer(text)
    tokens = [token.text for token in sp_tokens]
    trailing_whitespace = [bool(token.whitespace_) for token in sp_tokens]
    res = dg.labels(
        text=text, 
        tokens=sp_tokens, 
        street_address=street_address,
        student_name=student_name,
        username=username,
        personal_url=personal_url,
    )
    total = res.n_street_address + res.n_student_name + res.n_username + res.n_personal_url
    if total < min_entities:
        continue
    rows.append({
        "document": f"rh_{uuid.uuid4().hex}",
        "full_text": text,
        "tokens": tokens,
        "trailing_whitespace": trailing_whitespace,
        "labels": res.labels, 
        "n_street_address": res.n_street_address,
        "n_student_name": res.n_student_name,
        "n_username": res.n_username,
        "n_personal_url": res.n_personal_url,
        "street_address": street_address,
        "student_name": student_name,
        "username": username,
        "personal_url": personal_url,
    })


1870it [00:01, 1078.26it/s]


In [7]:
df = pd.DataFrame.from_records(rows)
cols = ["n_street_address", "n_student_name", "n_username", "n_personal_url"]
df[cols] = df[cols].astype(np.int8)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1654 entries, 0 to 1653
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   document             1654 non-null   object
 1   full_text            1654 non-null   object
 2   tokens               1654 non-null   object
 3   trailing_whitespace  1654 non-null   object
 4   labels               1654 non-null   object
 5   n_street_address     1654 non-null   int8  
 6   n_student_name       1654 non-null   int8  
 7   n_username           1654 non-null   int8  
 8   n_personal_url       1654 non-null   int8  
 9   street_address       1654 non-null   object
 10  student_name         1654 non-null   object
 11  username             1654 non-null   object
 12  personal_url         1654 non-null   object
dtypes: int8(4), object(9)
memory usage: 122.9+ KB


In [8]:
df.describe(percentiles=percentiles)

,n_street_address,n_student_name,n_username,n_personal_url
count,1654.000000,1654.000000,1654.000000,1654.000000
mean,1.641475,1.399033,0.662031,0.766022
std,0.707062,0.883621,0.906412,0.584355
min,0.000000,0.000000,0.000000,0.000000
1%,1.000000,0.000000,0.000000,0.000000
5%,1.000000,0.000000,0.000000,0.000000
10%,1.000000,1.000000,0.000000,0.000000
20%,1.000000,1.000000,0.000000,0.000000
30%,1.000000,1.000000,0.000000,0.000000
40%,1.000000,1.000000,0.000000,1.000000


In [9]:
for row in rows[:3]:
    print("===========")
    print(f"street_address={row['street_address']}")
    print(f"student_name={row['student_name']}")
    print(f"username={row['username']}")
    print(f"personal_url={row['personal_url']}")
    lbs = row["labels"]
    tokens = row["tokens"]
    ts = []
    for i in range(len(lbs)):
        if lbs[i]!="O":
            ts.append(tokens[i])
    print(ts)

street_address=Choa Chu Kang Avenue 4 (Block 444)
student_name=Rahman bin Hassan
username=_4807.rahman-iam--hassan
personal_url=periscope.tv/rahmanhassanxxx
['Choa', 'Chu', 'Kang', 'Avenue', '4', 'Rahman', 'bin', 'Hassan', '_', '4807.rahman', '-', 'iam', '--', 'hassan', '_', '4807.rahman', '-', 'iam', '--', 'hassan', 'Periscope.tv/rahmanhassanxxx', 'Rahman', 'bin', 'Hassan', '_', '4807.rahman', '-', 'iam', '--', 'hassan']
street_address=Jurong West Street 92 (Block 922)
student_name=Nicholas Sami
username=__iamNicholas_xxx.Sami
personal_url=blogspot.me/@iamnicholassami
['Jurong', 'West', 'Street', '92', '(', 'Block', '922', ')', '_', '_', 'iamNicholas_xxx', '.', 'Sami', 'Blogspot.me/@iamnicholassami']
street_address=Jalan Eunos (Eunos Mansion)
student_name=Nina Setiawan
username=the-nina3719setiawan
personal_url=sinaweibo.com/ninaiamsetiawanthe
['Jalan', 'Eunos', 'Nina', 'Setiawan', 'the', '-', 'nina3719setiawan', 'The', '-', 'nina3719setiawan', 'the', '-', 'nina3719setiawan', 'sinawei

In [10]:
%%time
with open("output/rh01.json", "w") as f:
    json.dump(rows, f)
df.to_parquet(f"output/rh01.parquet", index=False)
assert df.notna().all(axis=None)

CPU times: user 527 ms, sys: 0 ns, total: 527 ms
Wall time: 828 ms


In [11]:
tim.stop()
print(f"Total time taken {str(tim.elapsed)}")

Total time taken 0:00:02.940048
